In [1]:
from mxnet import gluon
import mxnet as mx
from mxnet import nd
from mxnet.gluon import nn 
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt
import PIL.Image

from gluoncv.data import VOCSegmentation
from gluoncv.utils.metrics.voc_segmentation import batch_pix_accuracy, batch_intersection_union

from mylib.deeplabv3p import DeepLabv3p

%matplotlib inline

/home/jiancheng/anaconda3/lib/python3.6/site-packages/urllib3/contrib/pyopenssl.py:46: DeprecationWarning: OpenSSL.rand is deprecated - you should use os.urandom instead
  import OpenSSL.SSL


In [15]:
s = set([3,4,5])

In [16]:
s.remove(3)

In [2]:
class CustomVOCSegmentation(VOCSegmentation):
    
    def __init__(self, split, size=(512,512)):
        super().__init__(split=split)
        self.size = size
        
    def __getitem__(self, index):
        img = PIL.Image.open(self.images[index]).convert('RGB')
        mask = PIL.Image.open(self.masks[index])
        img = img.resize(self.size, PIL.Image.BILINEAR)
        mask = mask.resize(self.size, PIL.Image.NEAREST)
        img = self._img_transform(img).transpose((2,0,1))/127.5 - 1.
        mask = self._mask_transform(mask)
        return img, mask

In [3]:
dataset = CustomVOCSegmentation(split='val', size=(480,480))

In [4]:
dataloader = gluon.data.DataLoader(dataset, batch_size=8)

In [5]:
model = DeepLabv3p(OS=8)
model.initialize(ctx=mx.gpu())
# weights = "/home/jiancheng/code/segmentation/deeplabv3p_gluon/tmp_weights/pascal_trainval/pascal_trainval.params"
weights = '/home/jiancheng/code/segmentation/deeplabv3p_gluon/tmp_weights/pascal_train_aug/pascal_train_aug.params'
model.load_params(filename=weights, ctx=mx.gpu())

In [6]:
tbar = tqdm(dataloader)
total_inter, total_union, total_correct, total_label = (0,)*4
for i, (x, y) in enumerate(tbar):
    x = x.copyto(mx.gpu())
    y = y.copyto(mx.gpu())
    pred = model(x)
    correct, labeled = batch_pix_accuracy(output=pred,target=y)
    inter, union = batch_intersection_union(output=pred,target=y,nclass=21)
    total_correct += correct.astype('int64')
    total_label += labeled.astype('int64')
    total_inter += inter.astype('int64')
    total_union += union.astype('int64')
    pix_acc = np.float64(1.0) * total_correct / (np.spacing(1, dtype=np.float64) + total_label)
    IoU = np.float64(1.0) * total_inter / (np.spacing(1, dtype=np.float64) + total_union)
    mIoU = IoU.mean()
    tbar.set_description('iter%s: pix_acc: %.4f, mIoU: %.4f' % (i, pix_acc, mIoU))

iter181: pix_acc: 0.9387, mIoU: 0.7381: 100%|██████████| 182/182 [04:53<00:00,  1.61s/it]


In [7]:
model = DeepLabv3p(OS=16)
model.initialize(ctx=mx.gpu())
# weights = "/home/jiancheng/code/segmentation/deeplabv3p_gluon/tmp_weights/pascal_trainval/pascal_trainval.params"
weights = '/home/jiancheng/code/segmentation/deeplabv3p_gluon/tmp_weights/pascal_train_aug/pascal_train_aug.params'
model.load_params(filename=weights, ctx=mx.gpu())

In [8]:
tbar = tqdm(dataloader)
total_inter, total_union, total_correct, total_label = (0,)*4
for i, (x, y) in enumerate(tbar):
    x = x.copyto(mx.gpu())
    y = y.copyto(mx.gpu())
    pred = model(x)
    correct, labeled = batch_pix_accuracy(output=pred,target=y)
    inter, union = batch_intersection_union(output=pred,target=y,nclass=21)
    total_correct += correct.astype('int64')
    total_label += labeled.astype('int64')
    total_inter += inter.astype('int64')
    total_union += union.astype('int64')
    pix_acc = np.float64(1.0) * total_correct / (np.spacing(1, dtype=np.float64) + total_label)
    IoU = np.float64(1.0) * total_inter / (np.spacing(1, dtype=np.float64) + total_union)
    mIoU = IoU.mean()
    tbar.set_description('iter%s: pix_acc: %.4f, mIoU: %.4f' % (i, pix_acc, mIoU))

iter181: pix_acc: 0.9249, mIoU: 0.6991: 100%|██████████| 182/182 [01:29<00:00,  2.04it/s]


In [9]:
model = DeepLabv3p(OS=8)
model.initialize(ctx=mx.gpu())
weights = "/home/jiancheng/code/segmentation/deeplabv3p_gluon/tmp_weights/pascal_trainval/pascal_trainval.params"
# weights = '/home/jiancheng/code/segmentation/deeplabv3p_gluon/tmp_weights/pascal_train_aug/pascal_train_aug.params'
model.load_params(filename=weights, ctx=mx.gpu())

In [10]:
tbar = tqdm(dataloader)
total_inter, total_union, total_correct, total_label = (0,)*4
for i, (x, y) in enumerate(tbar):
    x = x.copyto(mx.gpu())
    y = y.copyto(mx.gpu())
    pred = model(x)
    correct, labeled = batch_pix_accuracy(output=pred,target=y)
    inter, union = batch_intersection_union(output=pred,target=y,nclass=21)
    total_correct += correct.astype('int64')
    total_label += labeled.astype('int64')
    total_inter += inter.astype('int64')
    total_union += union.astype('int64')
    pix_acc = np.float64(1.0) * total_correct / (np.spacing(1, dtype=np.float64) + total_label)
    IoU = np.float64(1.0) * total_inter / (np.spacing(1, dtype=np.float64) + total_union)
    mIoU = IoU.mean()
    tbar.set_description('iter%s: pix_acc: %.4f, mIoU: %.4f' % (i, pix_acc, mIoU))

iter181: pix_acc: 0.9667, mIoU: 0.8395: 100%|██████████| 182/182 [04:45<00:00,  1.57s/it]


In [11]:
IoU.mean()

0.83952060808054962

In [12]:
np.float64(1.0) * total_inter / (np.spacing(1, dtype=np.float64) + total_union)

array([ 0.96210574,  0.86299041,  0.60878387,  0.84180434,  0.84475334,
        0.65800261,  0.87724081,  0.86911982,  0.91654737,  0.70522442,
        0.95383716,  0.83625859,  0.91960927,  0.89853939,  0.8757581 ,
        0.91294396,  0.67998638,  0.88289161,  0.75003861,  0.89003154,
        0.88346542])

In [13]:
mIoU

0.83952060808054962